In [12]:
import numpy as np
import scipy as sp

import pandas as pd

import matplotlib.pyplot as plt
pd.plotting.register_matplotlib_converters()
pd.options.mode.chained_assignment = None # suppress chained assignment warning

# Machine learning
from sklearn.model_selection import train_test_split as skl_ttsplit
import sklearn.decomposition as skl_dcmp
from sklearn.metrics import adjusted_rand_score

from pyclustering.cluster.kmedoids import kmedoids as KMedoids


import re

# Multiprocessing
from multiprocessing import Pool

## Getting Data

In [13]:
# Get data
price_data = pd.read_csv('../../data/sp500/sp500_since2000_prices.csv', index_col=0)
price_data.index = pd.to_datetime(price_data.index)
log_return_data = pd.read_csv('../../data/sp500/sp500_since2000_logreturns.csv', index_col=0)
log_return_data.index = pd.to_datetime(log_return_data.index)

# Get historical S&P 500 data
index_history_data = pd.read_csv('../../data/sp500/sp500_history.csv', index_col=0)
index_history_data.index  = pd.to_datetime(index_history_data.index)
index_history_log_returns = np.log(index_history_data['Close'] / index_history_data['Close'].shift(1))
index_history_log_returns_since2000 = index_history_log_returns['2000-01-01' :]

# Get company data
company_data = pd.read_csv('../../data/sp500/sp500_since2000_companies.csv')
company_data_convert_dict = {'ggroup': 'Int32', 'gind': 'Int32', 'gsector': 'Int32', 'gsubind': 'Int32', 'naics': 'Int32', 'sic': 'Int32', 'spcindcd': 'Int32', 'spcseccd': 'Int32'}
company_data = company_data.astype(company_data_convert_dict)
tic_lookup_by_gvkey = company_data.set_index('gvkey')[['tic']].to_dict()['tic']
gvkey_lookup_by_tic = company_data.set_index('tic')[['gvkey']].to_dict()['gvkey']

# Detrend data
log_return_data_detrended = pd.DataFrame(index = log_return_data.index, columns = log_return_data.columns)
for tic in log_return_data.columns:
    log_return_data_detrended[tic] = log_return_data[tic] -  index_history_log_returns_since2000

# Year
yearly_log_return_data = {}
yearly_log_return_data_detrended = {}
yearly_history_log_return_data = {}
yearly_history_log_return_vola = {}
for year in range(2000, 2021):
    yearly_log_return_data[year] = log_return_data['{}-01-01'.format(year) : '{}-12-31'.format(year)]
    yearly_log_return_data_detrended[year] = log_return_data_detrended['{}-01-01'.format(year) : '{}-12-31'.format(year)]
    yearly_history_log_return_data[year] = index_history_log_returns_since2000['{}-01-01'.format(year) : '{}-12-31'.format(year)]
    yearly_history_log_return_vola[year] = np.std(yearly_history_log_return_data[year])

In [6]:
# Import Hoberg Phillips reference data
hp_line_regex = r'(?P<gvkey>\d+)\s+(?P<year>\d+)\s+(?P<icode25>\d+)\s+(?P<icode50>\d+)\s+(?P<icode100>\d+)\s+(?P<icode200>\d+)\s+(?P<icode300>\d+)\s+(?P<icode400>\d+)\s+(?P<icode500>\d+)'

yearly_HP_cluster_labels = {}
fic_cluster_label = 'icode25'
with open('../../data/hp/fic_data.txt') as file:
    for line in file:
        # get cluster for each company for each year
        result = re.search(hp_line_regex, line)
        if result is not None:
            gvkey = int(result['gvkey'])
            year = int(result['year'])
            cluster_label = result[fic_cluster_label]
            if gvkey in tic_lookup_by_gvkey: # if the company is also present in our data
                if year not in yearly_HP_cluster_labels:
                    yearly_HP_cluster_labels[year] = pd.DataFrame(columns=['tic', 'cluster_label'])
                yearly_HP_cluster_labels[year] = yearly_HP_cluster_labels[year].append({'tic': tic_lookup_by_gvkey[gvkey], 'cluster_label': cluster_label}, ignore_index=True, )

In [7]:
yearly_HP_clusters = {}
for year in yearly_HP_cluster_labels:
    yearly_HP_clusters[year] = {}
    for label in yearly_HP_cluster_labels[year]['cluster_label'].unique():
        yearly_HP_clusters[year][label] = yearly_HP_cluster_labels[year][yearly_HP_cluster_labels[year]['cluster_label'] == label]['tic'].to_list()

## Matrix Generation

In [8]:
# Pearson correlation matrix and euclidean distance
yearly_correlations = {}
yearly_correlations_detrended = {}
yearly_average_correlation = {}
yearly_average_correlation_detrended = {}

yearly_euclidean = {}
yearly_euclidean_detrended = {}
yearly_available_companies = {} # companies for which data was available in this year

# corr calculates the correlation matrix, dropna removes lines and columns with only NaN in them
print('Calculating correlation matrices for ', end='')
for year in yearly_log_return_data:
    print('{}, '.format(year), end = '')
    
    yearly_correlations[year] = yearly_log_return_data[year].corr().dropna(how='all').dropna(how='all', axis=1)
    yearly_correlations_detrended[year] = yearly_log_return_data_detrended[year].corr().dropna(how='all').dropna(how='all', axis=1)
    
    yearly_average_correlation[year] = yearly_correlations[year].sum(axis=0).sum() / len(yearly_correlations[year])**2
    yearly_average_correlation_detrended[year] = yearly_correlations_detrended[year].sum(axis=0).sum() / len(yearly_correlations[year])**2
    
    yearly_euclidean[year] =  np.sqrt(2*(1-yearly_correlations[year] + 1e-15)).dropna(how='all').dropna(how='all', axis=1)
    yearly_euclidean_detrended[year] =  2-np.sqrt(2*(1-yearly_correlations_detrended[year] + 1e-15)).dropna(how='all').dropna(how='all', axis=1)
    
    yearly_available_companies[year] = yearly_correlations[year].columns.to_list()

print('Done!')

Calculating correlation matrices for 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, Done!


## Generating compatible output files

In [9]:
distmat_wildcard = r'sp500_yearly_distmat_{}.csv'
for year in yearly_correlations_detrended:
    # 1 - correlation for it to be a real distance
    (1 - yearly_correlations_detrended[year]).to_csv(distmat_wildcard.format(year))

In [10]:
avg_filename = r'sp500_yearly_avg.csv'
avg_detrended_filename = r'sp500_yearly_avg_detrended.csv'
avg_correlation_frame = pd.DataFrame.from_dict(yearly_average_correlation, orient = 'index', columns = ['avg_corr'])
avg_correlation_frame.index.name = 'year'
avg_correlation_detrended_frame = pd.DataFrame.from_dict(yearly_average_correlation_detrended, orient = 'index', columns = ['avg_corr'])
avg_correlation_detrended_frame.index.name = 'year'
avg_correlation_frame.to_csv(avg_filename)
avg_correlation_detrended_frame.to_csv(avg_detrended_filename)

In [18]:
vola_filename = r'sp500_yearly_vola.csv'
vola_frame = pd.DataFrame.from_dict(yearly_history_log_return_vola, orient = 'index', columns = ['vola'])
vola_frame.index.name = 'year'
vola_frame.to_csv(vola_filename)

In [11]:
hp_wildcard = r'../hp/hp_yearly_clusters_{}.csv'
for year in yearly_HP_cluster_labels:
    yearly_HP_cluster_labels[year].to_csv(hp_wildcard.format(year))